In [6]:
import pandas as pd
import numpy as np
import os
import datetime

In [7]:
def edit_column_date(frame,index):
    #Edits the date format of columns of dataframes
    #index: index of the first column of dates + 1
    i = 0
    for col in frame:
        i += 1
        if i >= index:
            new_d = date_format(col)
            frame = frame.rename(columns={col : new_d})
    return frame

In [8]:
def sort_dates(frame,index):
    #Sorts the columns by date of a frame with many nonconsecutive dates (several factors per date)
    Beg = list(frame.columns[:index]) #First four entries
    End = list(np.sort(np.array(frame.columns[index:]))) #Every Date Sorted
    cols = list(Beg + End) #Ordered Columns

    frame = frame[cols]
    return frame

In [9]:
def date_format(date):
    d = datetime.datetime.strptime(date, '%Y-%m-%d')
    return datetime.date.strftime(d, "%m/%d/%y")

In [10]:
#Loading in mobility data
DL_us_m50 = pd.read_csv('../../../../data/us/mobility/DL-us-m50.csv', encoding='latin1')
DL_us_m50_index = pd.read_csv('../../../../data/us/mobility/DL-us-m50_index.csv', encoding='latin1')
DL_us_samples = pd.read_csv('../../../../data/us/mobility/DL-us-samples.csv')

In [11]:
#Cleaning the datasets
DL_us_m50 = edit_column_date(DL_us_m50,6)
DL_us_m50_index = edit_column_date(DL_us_m50_index,6)
DL_us_samples = edit_column_date(DL_us_samples,6)

DL_us_m50 = DL_us_m50.drop(columns=['country_code','admin_level','admin1','admin2'])
DL_us_m50_index = DL_us_m50_index.drop(columns=['country_code','admin_level','admin1','admin2'])
DL_us_samples = DL_us_samples.drop(columns=['country_code','admin_level','admin1','admin2'])

In [12]:
#Separating data into state info
DL_us_m50_State = DL_us_m50[DL_us_m50.fips <= 1000]
DL_us_m50_index_State = DL_us_m50_index[DL_us_m50_index.fips <= 1000]
DL_us_samples_State = DL_us_samples[DL_us_samples.fips <= 1000]

In [13]:
#merging the 3 datasets together

Mobility_State = pd.merge(DL_us_m50_State, DL_us_m50_index_State, \
                    left_on='fips', right_on='fips', suffixes=('_M_m50', ''), sort=True)
Mobility_State = pd.merge(Mobility_State, DL_us_samples_State, \
                    left_on='fips', right_on='fips', suffixes=('_M_idx', '_M_samples'), sort=True)
#saving datasets with 3 values not consecutive and then consecutive
Mobility_State_Nonconsecutive = Mobility_State
Mobility_State_Consecutive = sort_dates(Mobility_State,1)
Mobility_State_Consecutive.to_csv('Mobility_State_Consecutive.csv')
Mobility_State_Nonconsecutive.to_csv('Mobility_State_Nonconsecutive.csv')